## A Python Implementation of Stochastic Gradient Hamiltonian Monte Carlo with example applications from simulated and real data

### Abstract

We implement the Stochastic gradient Hamiltonian Monte Carlo (SGHMC) algorithm described in the paper *Stochastic Gradient Hamiltonian Monte Carlo* by Chen et al. (2014). This particular Hamiltonian Monte Carlo (HMC) sampling method allows for efficient HMC sampling through the use of a noisy gradient term calculated using minibatches of the full data set. This mini-batching makes this method suitable for problems that would normally be computationally infeasible due to expensive gradient calculations on the full data set. We include both python and C++ implementations of the algorithm; a description of the changes made and performance gains in each successive version may be found in the **Optimization** section of this document. We include an example using a simulated noisy gradient and a known true target distribution, and an example using data drawn from a simulated two-component mixture of normals for which we sample the posterior of the mean parameters. There is an up-to-date GitHub repository for the code at <https://github.com/kelrenmor/sta663-FinalProject-SGHMC>. The package, including examples and the code used to generate results and figures for this docoment, is installable; instructions are available in the README file at the GitHub repository.

### Background

Hamiltonian Monte Carlo (HMC) sampling methods allow for proposing distant points in space with high acceptance probabilities. Hamiltonian Monte Carlo treats the probability density as a physical system being explored by a moving object. The object conserves energy overall, while trading between potential energy (in high probability regions) and momentum. HMC methods incorporate gradient information into the proposal distribution, which is especially advantageous for sampling a probability distribution with high correlations or many dimensions. This enables more efficient exploration of the space of interest than random walks, while still being in a Metropolis-Hastings framework.

The paper *Stochastic Gradient Hamiltonian Monte Carlo* by Chen et al. (2014) develops a variant of HMC that sidesteps the computational limitation of HMC methods that comes from having to compute the gradient when the data are large. Specifically, the authors use a stochastic gradient and introduce a friction term that allows the algorithm to maintain the desired target distribution and invariance properties.

This algorithm can be applied in any situation in which MCMC methods are needed, but is particularly useful in cases when the usual random walk methods tends to produce highly dependent samples (e.g., when there are high correlation between variables) or very low acceptance probabilities (e.g. when the sample space is very high dimensional). It also allows for scaling of Bayesian methods, with the use of minibatches of data.

Other developments to improve the flexibility of HMC include the "No U-Turn" sampler and Riemann manifold HMC. The authors argue that their Stochastic gradient Hamiltonian Monte Carlo (SGHMC) algorithm is an efficiency improvement to traditional HMC orthogonal to these other approaches, and thus could be combined with them to yield even better results.

### Description of algorithm

The Stochastic Gradient HMC algorithm is complex in its derivation, but simple in implementation. In words, the algorithm begins when the sampler is initialized. The algorithm draws a specified number of discrete samples by moving around the targeted probability distribution. In standard HMC each move is a deterministic function of the current location and momentum. In SGHMC, random noise is added to account for the stochastic estimation of the target distribution. In addition, during each move a friction term reduces the momentum for stability. Computationally, the algorithm is:


*Initialize ($\theta_0$, $v_0$, $\alpha$, $\eta$, $\hat{\beta}$)*

**for t = 1,2,... :**
    
$\qquad \theta_i = \theta_{i-1} + v$
    
$\qquad v = v - \eta \nabla \tilde{U(}x) - \alpha v + N(0, 2(\alpha - \hat{\beta}))$


**end **

The above computational algorithm represents the Hamiltonian dynamics, re-expressed in terms more familiar to Stochastic Gradient Descent. $\tilde{U}(x)$ is the minibatch approximation to the gradient, $\eta$ is the learning rate, $\alpha$ is the friction constant, and $\hat{\beta}$ is the approximation to the noise introduced by the stochastic gradient.

To go into more detail on how we perform mini-batching, the full data are split into `nbatches` random batches of size `batch_size` (defined by the user), at each iteration of the sampler. If there is not an even way to split the full data, the extra observations are thrown away in that epoch. The sampler runs on each batch, updating after each, until all batches have been explored. 

### Describe optimization for performance

Vectorization was performed in the original algorithm, so we don't have any optimization results for vectorized vs. non-vectorized Python versions of the sampler. Things we did to optimize performance relative to our original algorithm (see optimization_work.ipynb) include:
1. An algorithmic performance improvement in multivariate normal sampling. We obtain improvement through the use of Cholesky decomposition based multivariate normal sampling with pre-computation (outside of the main sampler loop) of the Cholesky decomposition of covariance matrices needed to sample from the relevant multivariate normals.
2. JIT compilation of our main SGHMC sampling algorithm (we tried the data batching code with and without JIT compilation for comparison within the JIT-compiled algorithm).
3. Re-writing the main algorithm, data batching, and gradient functions in C++ and using `pybind11` to wrap them.

|                            | Original     | Efficiency improvements | Numpy version (main algorithm only) | Numpy version (main algorithm and data batching) | C++ version   |
|----------------------------|--------------|-------------------------|-------------------------------------|--------------------------------------------------|---------------|
| Example 1 (Simulation)          | 23.5 (0.297) | 3.52 (0.091)            | 4.81 (0.465)                        |                                                  | 0.197 (0.003) |
| Mixture of Normals Example | 22.7 (0.144) | 21.7 (0.253)            | 22.0 (0.176)                        | 22.1 (0.062)                                     | 0.019 (0.000) |

The first row of Table 1 gives the time taken for the sampler to draw 50,000 samples from the target distribution of Example 1, a simulated noisy gradient described below. The second row of Table 1 gives the time taken for the sampler to draw 2,000 samples from the target in the mixtures of normals example, with 500 iterations over 4 size-50 data batches. Times shown are the mean (standard deviation) time in seconds across 7 runs. Tuning parameters are chosen to be the same as those described in the **Applications to simulated data sets** section below. Note that the deteriorated performance of the JIT-compiled version in the case Example 1 may be due to the fact that the data are not being resampled here (rather, artificial noise is being injected), so the overhead of JIT may outweigh the benefits. Since data batching are not performed, the second numpy version is the same as the first. JIT compilation seems to do little to help the mixture of normals example either. We note that the C++ version of the mixture of normals example yielded unstable behavior with the mixture example so it is possible we have a small bug in the data batching section of the C++ code, but computation times should still be relevant.

We see that the C++ version of the code is 11,839% faster (calculated as (old - new) / new x 100%) than the original version for Example 1 and 118,308% faster than the original for the mixtures of normals example. Profiling showed expensive computation times for Example 1 came from the multivariate normal sampling (which we fixed with the efficiency improvements noted above), and the sampler loop in general. Profiling showed expensive computation times for the mixture of normals model was the auto-gradient calculation done by the `jacobian` function we used from the `autograd` package in the Python version of the code. The JIT-compiled version was not able to improve as much relative to the original, likely because it could not automatically C++-ify these expensive computations, whereas explicitly coding the algorithm in C++ meant we wrote the gradient functions by hand.

If we were working with extremely large data sets, distributed computing would be an optimization avenue to explore. However, we used a small enough data example that local computation was sufficient.

As the algorithm is a MCMC sampler, parallelization is not possible because we have dependence within each chain. An option could be to run multiple chains and parallelize over the number of chains, but this would only give gains up to the number of chains run rather than improving the speed of the underlying algorithm itself. 

Note that our C++ optimization code was developed on the sta-663 server, so when we run it on our local machine we have some missing dependencies. However, we do include the `eigen3` library in our package. To replicate these results, one could download the optimization_work.ipynb notebook in the sta-663 environment.

### Applications to simulated data sets

We include two simulated examples with known target distributions from which the sampler should be drawing. The first, from the Chen et al. paper, samples from the target distribution with $U(\theta) = -2\theta^2 + \theta^4.$ Figure 1 shows the true distribution of $\theta,$ which is available in closed form.

![EMPTY_TEXT](figs/Example1_truth.png "TITLE")
<center>Figure 1: True distribution of $\theta$</center>

For the SGHMC we use artificially injected gradient noise; specifically, we set $\Delta \tilde{U}(\theta)=\Delta U(\theta) + \mathcal{N}(0,4).$ We set $\eta=0.001, \alpha=0.01, V=1,$ and sample for $50,000,000$ iterations using the C++ implementation of the algorithm. We see that in Figure 2a that our sampler finds the true modes of the posterior, but seems to concentrate too closely about these modes; the density is more peaked than the truth. We believe this phenomenon may be due to our choice of tuning parameters, but fiddling with these values often led to algorithm instability so we will leave this performance failure to be solved in future work. We note that the authors of the paper had success sampling from this example, so it is our implementation that is flawed rather than the algorithm.

![EMPTY_TEXT](figs/Example1_a.png "TITLE")
<center>Figure 2a: Univariate $\theta$ sampling with SGHMC</center>

The second example samples the mean parameters of a mixture of normals distribution. We assume we have data $x$ with $x \sim 0.5 \mathcal{N}(\mu_1,1) + 0.5 \mathcal{N}(\mu_2,1),$ i.e. a two-component mixture model with known variances and unknown means. We simulate $n=10,000$ data points drawn from each mixture component, with $\mu_1=-3$ and $\mu_2=3$. We set the priors on $\mu_1$ and $\mu_2$ both at independent $\mathcal{N}(0,10).$ We set $\eta=\frac{0.01}{n} \mathcal{I}, \alpha=0.1 \mathcal{I}, V=\mathcal{I},$ (we use $\mathcal{I}$ to denote the 2x2 identity matrix here) the batch size to 1,000 and sample for $100$ iterations using the Python implementation of the algorithm. The results are shown below in Figure 2b. Note that, as in our previous example, the algorithm seems to be focusing too precisely on the mode of the posterior, enough so that we are actually seeming to focus away from the MLE, even with a relatively diffuse prior. Also note that our algorithm hasn't picked up on the fact that we have two modes (due to the label swapping problem of our mixture model), likely because it doesn't stray from its initially found mode.

![EMPTY_TEXT](figs/MixNorm_a.png "TITLE")
<center>Figure 2b: Mixtures of normals sampling with SGHMC</center>

### Applications to real data sets

As we found the algorithm to be unstable and highly sensitive to the choice of tuning parameters even with our toy examples, we did not choose to test it on a real data set. The authors of the paper include an example of using the algorithm with a Bayesian Neural Net for classification, so this is one area in which the computational benefits of the sampler could potentially come in handy. 

### Comparative analysis with competing algorihtms

We compare the SGHMC method (as coded by us) to both the standard HMC method as implemented in the package `pyhmc`, and to Stan (a no-U-turn implementation of HMC) through the `pystan` package.

The results for the mixture of normals models are shown below. The `pyhmc` sampler exhibited difficulties even with these relatively simple examples. For the univariate sampling of $\theta$ in Example 1, we find that the sampler diverges and exhibits unstable behavior (the results below are for 500,000 samples). For the mixtures of normals example, if we begin the sampler at the true mixture component means, -3 and 3, we find that the algorithm does not explore the space at all; that is, we just get repeated samples of -3 and 3 as the output! When initialized elsewhere error messages were returned. Thus, we don't show any figures for the output of this sampler with our mixture of normals model. Another issue is that `pyhmc` is a very slow algorithm; 100 samples of Example 1 took over a minute to run. I would not choose to use this package again.

![EMPTY_TEXT](figs/Example1_b.png "TITLE")
<center>Figure 3: Univariate $\theta$ sampling with SGHMC</center>

For Stan, we do not choose an initial point, but let the package run using its defaults. For the Example 1 we set the number of chains to 4 and the number of iterations to 100,000, with the default number of burn-in samples. Stan captures the true density quite well. For the mixture of normals example we set the number of chains to 4 and the number of iterations to 1000, with the default number of burn-in samples. The most notable difference between our results and Stan for the mixture of normals example is that Stan finds both modes of the posterior (we have a label-swapping problem with our mixture model, which we don't address in our model formulation), and the credible intervals are much more realistic from Stan. Stan is a trusted, well-written and vetted algorithm and it shows; the sampled posterior is much closer to the true posterior than our algorithm, and without the need to fiddle with finicky tuning parameters.

![EMPTY_TEXT](figs/Example1_c.png "TITLE")
<center>Figure 4a: Univariate $\theta$ sampling with Stan</center>

![EMPTY_TEXT](figs/MixNorm_c.png "TITLE")
<center>Figure 4b: Mixtures of normals sampling with Stan</center>

### Discussion/conclusion

The strengths and weaknesses of the SGHMC algorithm have become very apparent through the implementation and testing process. The SGHMC algorithm is an extremely efficient sampler, as evidenced from testing. Optimizing the code has produced incredibly quick results on simulated datasets. However, the sampler includes a number of sensitive hyper parameters - the learning rate, the friction, and the estimation for the noise added due to stochastic minibatch sampling. If these are set incorrectly then the sampler is unstable. In simulated examples where the final answer is known, we have been successful in tuning these hyperparameters to create a robust sampler. However, in real-world situations where the truth is unknown, the basic SGHMC algorithm is insufficiently robust to be trusted. This matches the results that we have seen with the basic HMC package. Just using the default settings often produces unstable samples.

However, the highly developed STAN package uses the No U-Turn Sampler (NUTS) HMC algorithm, which is both efficient and stable without hyperparameter tuning. As noted in the original paper itself, there are a number of adaptive improvements that have been made to HMC - NUTS being one of them - which could be applied to the SGHMC algorithm. In fact, papers have already been written which fuse these ideas, such as in *A Complete Recipe for Stochastic Gradient MCMC*, which includes an algorithm for stochastic gradient Riemann HMC. As these improvements are incorporated, the efficiencies of SGHMC on large datasets will be more practical for real-world application.

### References/bibliography

Chen, T., Fox, E., and Guestrin, C. *Stochastic Gradient Hamiltonian Monte Carlo*. ArXiv e-prints. 1402.4102.

Ma Y.A., Chen, T., and Fox, E. *A Complete Recipe for Stochastic Gradient MCMC*. ArXiv e-prints. 1506.04696.


## Code

The GitHub repository for the write up and the package underlaying it may be found at <https://github.com/kelrenmor/sta663-FinalProject-SGHMC>. This is a public GitHub repository, with installation instructions in the README file. This report may be found in the paper directory. The package is in the SGHMC directory. This directory houses LICENSE.txt (an open source license), source code, test code, and examples. 

In order to download the repository and install the package, one should (in an environment where Python 3 is the default) type the following into the command line:

`$ git clone https://github.com/kelrenmor/sta663-FinalProject-SGHMC.git`

`$ cd sta663-FinalProject-SGHMC/SGHMC`

`$ python setup.py install`

To run examples, one can run the script in the 'run_examples.py' file.